In [17]:
!pip install beautifulsoup4

In [2]:
!pip install requests

In [18]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
from requests import get
import re

In [19]:
HEADERS={...}
drom_url = 'https://auto.drom.ru/region77/premium/'

#Отправим GET()-запрос на сайт и сохраним полученное в переменную 'page':

page = get(drom_url)

#Проверим подключение:

print(page.status_code)

#Код вернул нам статус код '200', мы успешно подключены



200


In [20]:
print(page.content)

b'<!DOCTYPE html>\n<html\n    xmlns="http://www.w3.org/1999/xhtml"\n    lang="ru"\n    xml:lang="ru"\n    class="drom-notouch no-js light no-main0"\n>\n<head>\n                    <title>\xd1\xef\xe5\xf6\xf0\xe0\xe7\xec\xe5\xf9\xe5\xed\xe8\xe5 \xe2 \xcc\xee\xf1\xea\xe2\xe5</title>\n        \n                <meta http-equiv="X-UA-Compatible" content="IE=Edge"/>\n\n        <meta name=\'copyright\' content=\'drom.ru\' />\n<meta name=\'theme-color\' content=\'#000000\' />\n<meta http-equiv=\'Content-Type\' content=\'text/html; charset=windows-1251\' />\n<meta charset=\'windows-1251\' />\n<meta name=\'candy.config\' content=\'{&quot;geor&quot;:77,&quot;geoc&quot;:0,&quot;id&quot;:27,&quot;b&quot;:1,&quot;bc&quot;:1,&quot;charset&quot;:&quot;utf-8&quot;,&quot;cf&quot;:{&quot;category_id&quot;:1}}\' />\n<meta name=\'description\' content=\'\xd7\xe0\xf1\xf2\xed\xfb\xe5 \xee\xe1\xfa\xff\xe2\xeb\xe5\xed\xe8\xff \xee \xef\xf0\xee\xe4\xe0\xe6\xe5 \xe0\xe2\xf2\xee\xec\xee\xe1\xe8\xeb\xe5\xe9 \xed\

In [21]:
#Воспользуемся BeautifulSoup4:
soup = BeautifulSoup(page.content, "html.parser")
soup

<!DOCTYPE html>

<html class="drom-notouch no-js light no-main0" lang="ru" xml:lang="ru" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Спецразмещение в Москве</title>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="drom.ru" name="copyright"/>
<meta content="#000000" name="theme-color"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta charset="utf-8"/>
<meta content='{"geor":77,"geoc":0,"id":27,"b":1,"bc":1,"charset":"utf-8","cf":{"category_id":1}}' name="candy.config"/>
<meta content="Частные объявления о продаже автомобилей на Drom.ru (Россия, Москва, Новосибирск, Владивосток и другие города)" name="description"/>
<meta content="авто, автомобили, форумы, автофорум, продажа б/у, пробег, подержанные, продажа авто, продажа автомобилей, отзывы, автовладельцы, объявления, цены, продажа, toyota, nissan, mitsubishi, honda, mazda, suzuki, subaru, audi, bmw, ford, opel, kia, hyundai, каталог" name="keywords"/>
<meta content="https://c.rdro

In [24]:
#Парсим по тэгу div чтобы получить данные о всех автомобилях на вкладке Спецразмещение в Москве
all=soup.find_all("div",  class_="css-12lf65q e3qgya0")

for item in all:
  name_car=item.find_all("span", class_="css-1t71ine e16zdi0i0") #Парсим Марки авто и год выпуска по тэгу span
  name_cars=[]
  for p in name_car:
    name_cars.append(p)
  df1 = pd.DataFrame(name_car, columns = ['Марка автомобиля и год выпуска'])
  df1

In [27]:
df1['Марка автомобиля']=df1['Марка автомобиля и год выпуска'].map(lambda x: str(x)[:-4])
df1['Год выпуска']=df1['Марка автомобиля и год выпуска'].map(lambda x: str(x)[-4:])
df1.drop('Марка автомобиля и год выпуска', axis= 1 , inplace= True )
df1


,Марка автомобиля,Год выпуска
0,Kia Sportage,2011
1,Chevrolet Aveo,2012
2,Hyundai Tucson,2023
3,Volkswagen ID.4,2023
4,Volkswagen ID.4,2022
5,Volkswagen ID.4,2023
6,Zeekr 001,2023
7,Ford Kuga,2020
8,Mercedes-Benz GLC,2023
9,Hyundai Santa Fe,2018


In [28]:
#Парсим Стоимость авто по тэгу div
cost_cars_all = []
for x in soup.find_all('div', class_='css-aimy81 egua2dj0'):
  # print(x)
# Очищаем данные
  x1 = re.sub("[^0-9]", "", str(x))
  # print(x1)
  x2=x1.replace("8120", "")
  x2_s= int(x2)

  cost_cars_all.append(x2_s)
df2 = pd.DataFrame(cost_cars_all, columns = ['Стоимость автомобиля'])
df2

,Стоимость автомобиля
0,1620000
1,510000
2,4450000
3,4523000
4,5400000
5,4450000
6,6970000
7,2820000
8,10199000
9,3669000


In [29]:
# объединяем DataFrames
df_full = pd.concat([df1, df2], join = 'outer', axis = 1)
print (df_full)

            Марка автомобиля Год выпуска  Стоимость автомобиля
0              Kia Sportage         2011               1620000
1            Chevrolet Aveo         2012                510000
2            Hyundai Tucson         2023               4450000
3           Volkswagen ID.4         2023               4523000
4           Volkswagen ID.4         2022               5400000
5           Volkswagen ID.4         2023               4450000
6                 Zeekr 001         2023               6970000
7                 Ford Kuga         2020               2820000
8         Mercedes-Benz GLC         2023              10199000
9          Hyundai Santa Fe         2018               3669000
10             BMW 7-Series         2017               6080000
11            Renault Logan         2008                400000
12      Chevrolet Silverado         2015               6999999
13         Hyundai Palisade         2023               7100000
14          Volkswagen ID.4         2023               